In [16]:
from algos.chadlor.live_trader import liveTrading
import json
import redis

In [2]:
lt = liveTrading()

In [17]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [19]:
coinbase_price = 500
bitmex_price = 500

In [20]:
parameters = json.load(open('algos/chadlor/parameters.json'))
lt = liveTrading()
current_pos, avgEntryPrice, pos_size = lt.get_position()
position_since = int(r.get('chadlor_position_since').decode())

if current_pos == "NONE":
    if coinbase_price > (parameters['bigger_than'] * bitmex_price):
        lt.fill_order('buy')
else:
    if position_since > parameters['position_since']:
        position_since = int(r.get('chadlor_position_since').decode())
        pnl_percentage = ((bitmex_price-avgEntryPrice)/avgEntryPrice) * 100 * parameters['mult']

        if pnl_percentage < parameters['loss_cap']:
            lt.fill_order('sell')

    if coinbase_price > (parameters['bigger_than'] * bitmex_price):
        r.set('chadlor_position_since', 0)

    if position_since >= parameters['position_since']:
        lt.fill_order('sell')

if current_pos == "NONE":
    r.set('chadlor_position_since', 0)
else:
    position_since = position_since + 1
    r.set('chadlor_position_since', position_since)

lt.set_position()

In [24]:
int(r.get('chadlor_position_since').decode())

0

In [22]:
current_pos

'NONE'

In [10]:
lt.get_position()

('NONE', 0, 0)

In [4]:
lt.get_balance()

0.00671262

In [5]:
lt.get_max_amount('buy')

(6, 47314.5)

In [6]:
lt.fill_order('buy')

Time at filling order is: 2021-02-27 20:14:35.716755
Sending market buy order for BTC/USD of size 6 on BITMEX in 2021-02-27 20:14:39.096697


In [7]:
lt.get_position()

('LONG', 47326.0, 6.0)

In [8]:
lt.fill_order('sell')

Time at filling order is: 2021-02-27 20:14:49.956703
Sending market sell order for BTC/USD of size 6.0 on BITMEX in 2021-02-27 20:14:52.154075


In [9]:
lt.get_position()

('NONE', 0, 0)